In [2]:
import os
import sys
import numpy as np
import tensorflow as tf
import utils
from time import time
import models as Models

Using TensorFlow backend.


In [3]:
def run(model, sess, dataset):
    avg_correct = 0.0
    for x, masks, y in dataset:
        input_feed = {
            model.inputs: np.array([i.numpy() for i in x]).T,
            model.masks: np.array([i.numpy() for i in masks]).T,
            model.labels: np.array([i.numpy() for i in y]).T
        }

        output_feed = [model.accuracy, model.op] if train else [model.accuracy, None]
        num_correct, _ = sess.run(output_feed, input_feed)
        avg_correct += num_correct
    avg_correct /= len(dataset)
    return avg_correct

In [4]:
NUM_EPOCHS = 2
BATCH_SIZE = 500
SEQ_LEN = 500
VOCAB_SIZE = 500
train = True

train_data, test_data, vocab = utils.get_datasets(batch_size=BATCH_SIZE,num_words=VOCAB_SIZE,seq_len=SEQ_LEN)
embeddings = utils.get_embeddings(vocab, './glove.6B/glove.6B.300d.txt')

model = Models.DenseQRNNModel(embeddings, BATCH_SIZE, SEQ_LEN, VOCAB_SIZE, train)
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ckpt_path = os.path.join('checkpoints', 'dense')
    utils.check_restore_parameters(sess, saver, ckpt_path)
    epoch = model.epoch.eval()

    for i in range(epoch, NUM_EPOCHS):
        train_acc = run(model, sess, train_data)
        print ('epoch', i, 'average train acc:', train_acc)
        sess.run(tf.assign(model.epoch, i+1))

    saver.save(sess, ckpt_path)

    test_acc = run(model, sess, test_data)
    print ('average test acc:', test_acc)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Initializing fresh parameters
epoch 0 average train acc: 0.5254000240564346
epoch 1 average train acc: 0.5692000222206116
average test acc: 0.5791600298881531
